In [1]:
import os
os.chdir('../')
from google.cloud import vision
import sys

from detect_text import any_text_triggering
from places import get_places, get_place_details, load_zipcodes
from streetview import get_streetview_image
from store import Store
from vision import extract_image_text


In [2]:
from main import load_text

In [3]:
zipcodes = load_text('louisville_metro_zipcodes.txt')
store_types = load_text('store_types.txt')
trigger_phrases = load_text('trigger_phrases.txt')
images_dir = 'streetview_images'

In [4]:
## 
# Single Select
store_types = ['tobacco']
zipcodes = [40214]

In [21]:
stores = []
for zipcode in zipcodes:
    for store_type in store_types:
        query = f'{store_type}+in+{zipcode}'
        for place in get_places(query):
            stores.append(Store(place))

In [29]:
details

{'reviews': [{'author_name': 'Jennifer Carroll',
   'author_url': 'https://www.google.com/maps/contrib/106421652499158140864/reviews',
   'language': 'en',
   'original_language': 'en',
   'profile_photo_url': 'https://lh3.googleusercontent.com/a-/ALV-UjXSETED82zOmyWocxncdkC_ajJIql5y_8qyZtF5-De9gCE=s128-c0x00000000-cc-rp-mo-ba4',
   'rating': 5,
   'relative_time_description': '2 years ago',
   'text': 'Friendly and very helpful staff! Large selection of items and very reasonable prices. Will definitely visit again and also recommend to anyone I know.',
   'time': 1621715682,
   'translated': False},
  {'author_name': 'John Burkhead',
   'author_url': 'https://www.google.com/maps/contrib/104834117798754362960/reviews',
   'profile_photo_url': 'https://lh3.googleusercontent.com/a-/ALV-UjVoNyRglgnN26OG7y-To76jv5qpKpqoI855ioK41yz43X8=s128-c0x00000000-cc-rp-mo-ba3',
   'rating': 4,
   'relative_time_description': '8 months ago',
   'text': '',
   'time': 1673679936,
   'translated': False}

In [33]:
for store in stores:
    details = get_place_details(store.place['place_id'])
    if details is None:
        if args.verbose:
            print('ERROR: Failed to retrieve details for place', store.place['place_id'])
        continue
    store.photos = details['photos'] if 'photos' in details else []
    store.reviews = details['reviews'] if 'reviews' in details else []
    if 'website' in details:
        store.website = details['website']

In [42]:
for store in stores:
    review_images_dir = f"{images_dir}/{store.place['place_id']}"
    if not os.path.isdir(review_images_dir):
        os.mkdir(review_images_dir)
    for photo in store.photos:
        image_review = get_place_photo(photo['photo_reference'])
        image_path = '{review_images_dir}/{photo_id}.jpg'.format(
                review_images_dir = review_images_dir,
                photo_id=photo['photo_reference'])
        with open(image_path, 'wb') as f:
            f.write(image_review)
        store.review_images = review_images_dir


In [38]:
#details['photos']
import requests
def get_place_photo(photo_id, max_width = 640):
    api_key = os.getenv('MAPS_API_KEY')
    base_url = f'https://maps.googleapis.com/maps/api/place/photo?maxwidth={max_width}'
    url = f'{base_url}&photo_reference={photo_id}&key={api_key}'
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f'ERROR: request to Maps Places API failed with status code {resp.status_code}')
    return resp.content
